In [169]:
# imports
import requests as r
import json
import pandas as pd
import numpy as np
import itertools

from pprint import pprint
from io import BytesIO

In [64]:
def get_floors(contract_address=None):
    # 0x329fd5e0d9aad262b13ca07c87d001bec716ed39
    accumulator = []
    url = "https://api.opensea.io/api/v1/assets"
    empty = False
    offset = 0
    
    while empty == False:
        params = {"asset_contract_address":contract_address,"order_direction":"desc","offset":offset,"limit":"50"}
        response = r.get(url, params=params)    
        data = json.loads(response.text)
        accumulator.append(data['assets'])
        if len(data['assets']) < 1: 
            empty = True        
        
        offset += 50
    
    return accumulator     

In [149]:
def flatten_json(big_blob):
    accumulator = []
    
    for blob in big_blob:
        token_id = blob['token_id']
        num_sales = blob['num_sales']
        
        if blob['sell_orders'] != None:        
            sale_kind = blob['sell_orders'][0]['sale_kind']
            listing_time = blob['sell_orders'][0]['listing_time']
            current_price = round(np.float(blob['sell_orders'][0]['current_price'])) / 1000000000000000000.        
        else:
            sale_kind = None
            listing_time = None
            current_price = None 
        
        accumulator.append([token_id, num_sales, sale_kind, listing_time, current_price])
    
    return accumulator                        

In [146]:
json_blob = get_floors('0x329fd5e0d9aad262b13ca07c87d001bec716ed39')
single_blob = list(itertools.chain(*json_blob))

In [226]:
_df = flatten_json(single_blob)
df = pd.DataFrame(_df, columns=['token_id','num_sales','sale_kind','listing_time','current_price'])

# fix data types
df.token_id = df.token_id.astype('int64', copy=False)
df.listing_time = pd.to_datetime(df.listing_time,unit='s')
df['data_created'] = pd.to_datetime('now')

In [223]:
# grab latest test from timshel's spreadsheet

r = requests.get('https://docs.google.com/spreadsheet/ccc?key=1PCaVj5GOwwY6MFK6ZxqZRkrT_XYcW2b2iscJHE8ATOU&output=csv&gid=761760938')
rare_data = r.content    
rare_df = pd.read_csv(BytesIO(rare_data), index_col=0)
rare_df.columns = rare_df.iloc[0]
rare_df = rare_df[1:]
rare_df['id']  = rare_df.id.astype('int64', copy=False)
rare_df = rare_df.iloc[: , :26]

In [256]:
joined_df = rare_df.set_index('id').join(df.set_index('token_id'))
joined_df['opensea_link'] = joined_df.reset_index().id.apply(lambda x: 'https://opensea.io/assets/0x329fd5e0d9aad262b13ca07c87d001bec716ed39/'+str(x))

# for currently on sale
joined_df = joined_df[:][joined_df['current_price'].notnull()]
joined_df.to_csv('output.csv')

In [257]:
joined_df

,"Product (implied rarity, low is good)",New Rarity Approach\n\nUsing Rarity.tools method,"Rarity score (low is good), using Product (which is NOT a perfect way to get rarity)","Sum (implied rarity, low is good)","Rarity score (low is good), using Sum (which is NOT a good way to do this haha)",Can FILTER TO SPECIAL ONLY,Top Rarity (Using Rarity Tools Method),Top Rarity (Using Product Method),1:1s,1:10s,...,Dragon,Demon,Phoenix,New Rarity Approach\n\nUsing Rarity.tools,num_sales,sale_kind,listing_time,current_price,data_created,opensea_link
id,,,,,,,,,,,,,,,,,,,,,
927,1152363.99992,32,21,39361,19,Special,>200 Rarity,>200 Rarity,>4 1:1s,NaN,...,2,1,1,"26,612",1,0.0,2021-09-21 18:25:11,7.000000,2021-09-23 19:03:50.319395,https://opensea.io/assets/0x329fd5e0d9aad262b1...
2459,82256649828375200000000000.00000,34,905,82189,1970,Special,>200 Rarity,NaN,>4 1:1s,NaN,...,1,0,0,"26,403",0,0.0,2021-09-23 17:45:26,0.150000,2021-09-23 19:03:50.319395,https://opensea.io/assets/0x329fd5e0d9aad262b1...
3903,8259233278871770000000000000.00000,51,1156,81156,1872,Special,>200 Rarity,NaN,>4 1:1s,NaN,...,0,0,0,"25,054",0,1.0,2021-09-21 19:15:53,1.019563,2021-09-23 19:03:50.319395,https://opensea.io/assets/0x329fd5e0d9aad262b1...
1050,14733934927495600000000000000000000.00000,52,2068,97420,3256,Special,>200 Rarity,NaN,>4 1:1s,NaN,...,0,0,0,"25,024",1,0.0,2021-09-22 13:09:06,1.000000,2021-09-23 19:03:50.319395,https://opensea.io/assets/0x329fd5e0d9aad262b1...
3821,4964606155664550.00000,58,179,61601,458,Special,>200 Rarity,>200 Rarity,NaN,NaN,...,0,0,0,"24,799",1,0.0,2021-09-21 21:40:13,5.000000,2021-09-23 19:03:50.319395,https://opensea.io/assets/0x329fd5e0d9aad262b1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255,1793948339256850000000000000000000000000000000...,3972,3959,119664,3937,NaN,NaN,NaN,NaN,NaN,...,0,0,0,"1,661",0,0.0,2021-09-21 19:22:07,0.340000,2021-09-23 19:03:50.319395,https://opensea.io/assets/0x329fd5e0d9aad262b1...
713,1800682164230860000000000000000000000000000000...,3976,3909,109008,3764,NaN,NaN,NaN,NaN,NaN,...,0,1,0,"1,505",1,0.0,2021-09-22 20:47:05,0.333000,2021-09-23 19:03:50.319395,https://opensea.io/assets/0x329fd5e0d9aad262b1...
1183,2017058286975130000000000000000000000000000000...,3978,3960,111170,3806,NaN,NaN,NaN,NaN,NaN,...,0,0,0,"1,463",0,0.0,2021-09-21 19:20:42,0.388800,2021-09-23 19:03:50.319395,https://opensea.io/assets/0x329fd5e0d9aad262b1...
